In [1]:
import sys
sys.path.append("..")

## Load Dataset

In [2]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("jtz18/skin-lesion")

c:\Users\jonta\code\school\t8\DeepLearning\50.039-DL\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\jonta\code\school\t8\DeepLearning\50.039-DL\venv\lib\site-packages\datasets\load.py:1461: FutureWarning: The repository for jtz18/skin-lesion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/jtz18/skin-lesion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


## Seeding

In [3]:
import torch
import numpy as np
import random
import os

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)

  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

set_seed(42)

## Init Wandb

In [4]:
import wandb

# Define sweep config
sweep_configuration = {
    "name": "unet-sweep",
    "method": "bayes",
    "metric": {"goal": "maximize", "name": "Test Dice Score"},
    "parameters": {
        "learning_rate": {"values": [0.0001, 0.001, 0.01]},
        "batch_size": {"values": [4, 8, 16, 32, 64]},
        "epochs": {"values": [3]},
        "dropout_rate": {"values": [0, 0.1, 0.2, 0.3]},
        # "embedding_and_heads": {"values": [(256, 8), (512, 8), (512, 16), (768, 12), (1024, 16), (1024, 32), (2048, 32)]},
        "kernel_size": {"values": [3, 5, 7]},
    },
}

# Initialize sweep by passing in config.
sweep_id = wandb.sweep(sweep=sweep_configuration, project="unet-skin-lesion")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: qjjguvd6
Sweep URL: https://wandb.ai/jooz-cave/unet-skin-lesion/sweeps/qjjguvd6


In [5]:
# Hyperparameters etc.
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

DEVICE = device

NUM_WORKERS = 0

IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256

PIN_MEMORY = True
LOAD_MODEL = False
CHECKPOINT_FILENAME = ""   # Model checkpoint filename if LOAD is True eg: checkpoints/checkpoint_9.pth.tar or None
CLASS = "task1"

# Define Training Function

In [6]:
from tqdm import tqdm
def train_fn(loader, model, optimizer, loss_fn, scaler):
    loop = tqdm(loader)

    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=DEVICE)
        targets = targets.float().unsqueeze(1).to(device=DEVICE)

        # forward
        with torch.cuda.amp.autocast():
            predictions = model(data)
            loss = loss_fn(predictions, targets)
            wandb.log({"Training Loss": loss})

        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # update tqdm loop
        loop.set_postfix(loss=loss.item())

## Augmentations

In [7]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch import nn
from src.utils import *


train_transform = A.Compose(
    [
        A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
        A.Rotate(limit=35, p=1.0),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.1),
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ],
)

val_transforms = A.Compose(
    [
        A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ],
)

## Training Loop

In [8]:
# from src.segFormer import Segformer
from src.segFormer2 import SegFormer
from src.maluNet import MALUNet
from src.unet import UNET
from src.unetr import UNETR
from src.utils.utils import *

def main():
    run = wandb.init()

    # note that we define values from `wandb.config`
    # instead of defining hard values
    LEARNING_RATE = wandb.config.learning_rate
    BATCH_SIZE = wandb.config.batch_size
    NUM_EPOCHS = wandb.config.epochs
    DROPOUT_RATE = wandb.config.dropout_rate

    # model = Segformer(dropout=DROPOUT_RATE).to(device)
    model = UNET(dropout_rate=DROPOUT_RATE, kernel_size=wandb.config.kernel_size).to(device)
    # model = MALUNet(dropout_rate=DROPOUT_RATE).to(device)
    # model = UNETR(dropout=DROPOUT_RATE, embed_dim=wandb.config.embedding_and_heads[0], num_heads=wandb.config.embedding_and_heads[1]).to(device)
    # model = SegFormer(
    # in_channels=3,
    # widths=[64, 128, 256, 512],
    # depths=[3, 4, 6, 3],
    # all_num_heads=[1, 2, 4, 8],
    # patch_sizes=[7, 3, 3, 3],
    # overlap_sizes=[4, 2, 2, 2],
    # reduction_ratios=[8, 4, 2, 1],
    # mlp_expansions=[4, 4, 4, 4],
    # decoder_channels=256,
    # scale_factors=[8, 4, 2, 1],
    # num_classes=1,
    # drop_prob=DROPOUT_RATE,
    # ).to(device)
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    wandb.log({"MODEL": model.__class__.__name__})


    train_loader, val_loader, test_loader = get_loaders(
        dataset,
        BATCH_SIZE,
        train_transform,
        val_transforms,
        NUM_WORKERS,
        PIN_MEMORY,
    )

    start_epoch = 0
    if LOAD_MODEL:
        checkpoint = torch.load(CHECKPOINT_FILENAME)
        model, optimizer, start_epoch = load_checkpoint(checkpoint, model, optimizer)



    check_accuracy(val_loader, model, device=DEVICE, loss_fn=loss_fn)
    scaler = torch.cuda.amp.GradScaler()

    for epoch in range(start_epoch, NUM_EPOCHS):
        print(f"epoch: {epoch}")
        wandb.log({"epoch": epoch})
        train_fn(train_loader, model, optimizer, loss_fn, scaler)

        # save model
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer":optimizer.state_dict(),
            "epoch": epoch,
        }
        save_checkpoint(checkpoint, filename=f"checkpoint_{epoch}.pth.tar")

        # check accuracy
        check_accuracy(val_loader, model, device=DEVICE, loss_fn=loss_fn)

        # print some examples to a folder
        save_predictions_as_imgs(
            val_loader, model, folder=f"saved_images/{epoch}", device=DEVICE
        )

    # Final Evaluation Test
    check_accuracy(test_loader, model, device=DEVICE, loss_fn=loss_fn, mode="test")
    run.finish()

torch.Size([1, 1, 224, 224])


In [9]:
wandb.agent(sweep_id, function=main)

wandb: Agent Starting Run: cidmfzau with config:
wandb: 	batch_size: 8
wandb: 	dropout_rate: 0.3
wandb: 	epochs: 3
wandb: 	kernel_size: 5
wandb: 	learning_rate: 0.01
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jtz18 (jooz-cave). Use `wandb login --relogin` to force relogin


Validation Loss: 0.8416433563599219
Got 1840129/6553600 with acc 28.08
Dice score: 0.4170222580432892
epoch: 0


100%|██████████| 325/325 [07:16<00:00,  1.34s/it, loss=nan]  


=> Saving checkpoint
Validation Loss: nan
Got 4795034/6553600 with acc 73.17
Dice score: 0.0
epoch: 1


100%|██████████| 325/325 [07:15<00:00,  1.34s/it, loss=nan]


...

=> Saving checkpoint
Validation Loss: 0.6537922620773315
Got 5985392/6553600 with acc 91.33
Dice score: 0.8220670223236084
Test Loss: 0.6559141626358033
Got 58334183/65536000 with acc 89.01
Dice score: 0.7750762104988098


Dice Score,▁███
Pixel Accuracy,▁███
Test Dice Score,▁
Test Loss,▁
Test Pixel Accuracy,▁
Training Loss,▆▅▅▅▅▄▄▃▄▄▅▄▅▅▄▃▄▂▄▃▅▃▃▂▃▂█▃▂▅▄▂▄▂▁▁▂▃▁▁
Validation Loss,█▂▁▁
epoch,▁▅█
Dice Score,0.82207
MODEL,UNET
Pixel Accuracy,91.32983


wandb: Agent Starting Run: mp89d6ga with config:
wandb: 	batch_size: 16
wandb: 	dropout_rate: 0.1
wandb: 	epochs: 3
wandb: 	kernel_size: 7
wandb: 	learning_rate: 0.0001
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Validation Loss: 0.8490892137799945
Got 1758566/6553600 with acc 26.83
Dice score: 0.41082340478897095
epoch: 0


 26%|██▋       | 43/163 [02:03<06:08,  3.07s/it, loss=0.388]